# Video Game Genre Prediction

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import utils

import os, PIL
from glob import glob

import tensorflow as tf

print(tf.__version__)

2.6.0


In [2]:
""" Sequential Model Architecture """
Sequential = tf.keras.models.Sequential

""" Data Preprocessing Functions """
Resizing = tf.keras.layers.experimental.preprocessing.Resizing
Rescaling = tf.keras.layers.experimental.preprocessing.Rescaling

""" Data Augmentation Functions """
RandomFlip = tf.keras.layers.experimental.preprocessing.RandomFlip
RandomRotation = tf.keras.layers.experimental.preprocessing.RandomRotation
RandomZoom = tf.keras.layers.experimental.preprocessing.RandomZoom

""" Artificial Neural Network Layer Inventory """
Dense = tf.keras.layers.Dense
Dropout = tf.keras.layers.Dropout

""" Convolutional Neural Network Layer Inventory """
Conv2D = tf.keras.layers.Conv2D
MaxPool2D = tf.keras.layers.MaxPool2D
Flatten = tf.keras.layers.Flatten

""" Residual Network Layer Inventory """
ResNet50 = tf.keras.applications.resnet50.ResNet50

""" Function to Load Images from Target Folder """
image_dataset_from_directory = tf.keras.preprocessing.image_dataset_from_directory

In [3]:
DATA_DIRECTORY = "dataset/"
FIGHTING_GENRE_COVERS = f"{DATA_DIRECTORY}/fighting/*"
INDIE_GENRE_COVERS = f"{DATA_DIRECTORY}/indie/*"
PLATFORM_GENRE_COVERS = f"{DATA_DIRECTORY}/platform/*"
PUZZLE_GENRE_COVERS = f"{DATA_DIRECTORY}/puzzle/*"
SPORT_GENRE_COVERS = f"{DATA_DIRECTORY}/indie/*"

print(f"{len(glob(FIGHTING_GENRE_COVERS))} Fighting Game Covers")
print(f"{len(glob(INDIE_GENRE_COVERS))} Indie Game Covers")
print(f"{len(glob(PLATFORM_GENRE_COVERS))} Platform Game Covers")
print(f"{len(glob(PUZZLE_GENRE_COVERS))} Puzzle Game Covers")
print(f"{len(glob(SPORT_GENRE_COVERS))} Sport Game Covers")

1000 Fighting Game Covers
1000 Indie Game Covers
1000 Platform Game Covers
1000 Puzzle Game Covers
1000 Sport Game Covers


In [4]:
batch_size = 32
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

train = image_dataset_from_directory(
    directory=DATA_DIRECTORY,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=batch_size
)

validation = image_dataset_from_directory(
    directory=DATA_DIRECTORY,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=batch_size
)

class_names = train.class_names
print(class_names)

Found 5000 files belonging to 5 classes.
Using 4000 files for training.
Found 5000 files belonging to 5 classes.
Using 1000 files for validation.
['fighting', 'indie', 'platform', 'puzzle', 'sport']


In [6]:
train = train.cache().shuffle(4000).prefetch(buffer_size=tf.data.AUTOTUNE)
validation = validation.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [7]:
input_tensor = tf.keras.Input(shape=(224,224,3))
resnet_model = ResNet50(include_top=False, weights="resnet50_weights.h5", input_tensor=input_tensor)

In [8]:
for layer in resnet_model.layers[:143]:
    layer.trainable = False

for i, layer in enumerate(resnet_model.layers):
    print(i, layer.name, "Trainable: ", layer.trainable)

0 input_1 Trainable:  False
1 conv1_pad Trainable:  False
2 conv1_conv Trainable:  False
3 conv1_bn Trainable:  False
4 conv1_relu Trainable:  False
5 pool1_pad Trainable:  False
6 pool1_pool Trainable:  False
7 conv2_block1_1_conv Trainable:  False
8 conv2_block1_1_bn Trainable:  False
9 conv2_block1_1_relu Trainable:  False
10 conv2_block1_2_conv Trainable:  False
11 conv2_block1_2_bn Trainable:  False
12 conv2_block1_2_relu Trainable:  False
13 conv2_block1_0_conv Trainable:  False
14 conv2_block1_3_conv Trainable:  False
15 conv2_block1_0_bn Trainable:  False
16 conv2_block1_3_bn Trainable:  False
17 conv2_block1_add Trainable:  False
18 conv2_block1_out Trainable:  False
19 conv2_block2_1_conv Trainable:  False
20 conv2_block2_1_bn Trainable:  False
21 conv2_block2_1_relu Trainable:  False
22 conv2_block2_2_conv Trainable:  False
23 conv2_block2_2_bn Trainable:  False
24 conv2_block2_2_relu Trainable:  False
25 conv2_block2_3_conv Trainable:  False
26 conv2_block2_3_bn Trainable: 

In [9]:
resizing_layer = Resizing(IMAGE_HEIGHT, IMAGE_WIDTH)
normalization_layer = tf.keras.layers.BatchNormalization()

input_layer = tf.keras.layers.InputLayer(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

flatten_layer = Flatten()

dropout_layer = Dropout(0.5)

dense_layer_1 = Dense(256, activation="relu")
dense_layer_2 = Dense(128, activation="relu")
dense_layer_3 = Dense(64, activation="relu")
output_layer = Dense(1, activation="sigmoid")

model = Sequential()

model.add(input_layer)
model.add(resizing_layer)
model.add(resnet_model)
model.add(flatten_layer)
model.add(normalization_layer)
model.add(dense_layer_1)
model.add(dropout_layer)
model.add(dense_layer_2)
model.add(dropout_layer)
model.add(dense_layer_3)
model.add(dropout_layer)
model.add(output_layer)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 100352)            401408    
_________________________________________________________________
dense (Dense)                (None, 256)               25690368  
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3

In [ ]:
checkpoint = tf.keras.callbacks(filepath="model/genre_model.h5",
                                monitor="val_accuracy",
                                mode="max",
                                save_best_only=True)

In [10]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [11]:
history = model.fit(train, validation_data=validation, callbacks=[checkpoint], epochs=5)

Epoch 1/5
125/125 [==============================] - 420s 3s/step - loss: 0.0000e+00 - accuracy: 0.2030 - val_loss: 0.0000e+00 - val_accuracy: 0.1910
Epoch 2/5
125/125 [==============================] - 387s 3s/step - loss: 0.0000e+00 - accuracy: 0.2023 - val_loss: 0.0000e+00 - val_accuracy: 0.1910
Epoch 3/5
125/125 [==============================] - 386s 3s/step - loss: 0.0000e+00 - accuracy: 0.2023 - val_loss: 0.0000e+00 - val_accuracy: 0.1910
Epoch 4/5
125/125 [==============================] - 392s 3s/step - loss: 0.0000e+00 - accuracy: 0.2023 - val_loss: 0.0000e+00 - val_accuracy: 0.1910
Epoch 5/5
125/125 [==============================] - 431s 3s/step - loss: 0.0000e+00 - accuracy: 0.2023 - val_loss: 0.0000e+00 - val_accuracy: 0.1910


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 100352)            401408    
_________________________________________________________________
dense (Dense)                (None, 256)               25690368  
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3

In [13]:
model.evaluate(validation)

32/32 [==============================] - 60s 2s/step - loss: 0.0000e+00 - accuracy: 0.1910


[0.0, 0.19099999964237213]

In [15]:
model.save("model/genre_model.h5")

/Users/kou/dev/courses/acs3520/Fire-Detection-API-Project/env/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
